In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/participants_dataset.csv")
data

,ID,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,label
0,452,steven lux,steven,lux,2013-01-05,Male,1953-06-15,62,Greater than 45,Caucasian,...,1,Low,2013-01-05,2014-03-25,2014-03-26,0,2,444,0,0.0
1,573,andre small,andre,small,2013-04-03,Male,1987-10-01,28,25 - 45,African-American,...,3,Low,2013-04-03,2013-04-02,2013-04-04,1,1,1094,0,0.0
2,4952,willie gray,willie,gray,2014-10-31,Male,1959-01-12,57,Greater than 45,African-American,...,2,Low,2014-10-31,2014-10-31,2014-10-31,8,0,265,1,1.0
3,3521,nickson marcellus,nickson,marcellus,2014-01-23,Male,1996-07-11,19,Less than 25,African-American,...,10,High,2014-01-23,2016-01-05,2016-02-26,1,0,711,1,1.0
4,369,patria barnes,patria,barnes,2013-12-09,Female,1978-06-06,37,25 - 45,Other,...,1,Low,2013-12-09,2013-12-08,2013-12-09,0,0,844,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,2361,cindy stillman,cindy,stillman,2013-08-11,Female,1967-01-06,49,Greater than 45,Caucasian,...,1,Low,2013-08-11,2013-08-10,2013-08-22,2,11,245,1,NaN
7210,1865,mario brown,mario,brown,2013-11-20,Male,1979-11-24,36,25 - 45,Caucasian,...,1,Low,2013-11-20,2013-11-19,2013-12-14,0,24,863,0,NaN
7211,4086,andre barrington,andre,barrington,2014-01-24,Male,1989-06-12,26,25 - 45,African-American,...,10,High,2014-01-24,2014-05-04,2014-05-09,13,7,100,0,NaN
7212,4569,tyson harper,tyson,harper,2013-08-29,Male,1978-10-02,37,25 - 45,Caucasian,...,1,Low,2013-08-29,2014-07-18,2014-12-02,5,0,323,0,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataX = data.drop('label', axis=1)
dataY = data["label"]

In [ ]:
# remove unnecessary data
# cols_to_keep = ["is_recid", "event", "end", "r_charge_degree", "r_offense_date", "r_jail_in", "r_jail_out", "r_charge_desc"]
cols_to_keep = ["compas_screening_date", "screening_date", "decile_score", "r_offense_date", "vr_charge_degree", "decile_score", "r_charge_desc", "vr_offense_date", "priors_count", "is_recid", "r_jail_in", "vr_charge_desc", "r_jail_out", "r_charge_degree", "is_violent_recid", "v_screening_date", "priors_count.1", "end", "event"]
for col in dataX.columns:
    if not col in cols_to_keep:
        dataX = dataX.drop(col, axis=1)
    


In [ ]:
dataX.columns

Index(['compas_screening_date', 'decile_score', 'priors_count', 'is_recid',
       'r_charge_degree', 'r_offense_date', 'r_charge_desc', 'r_jail_in',
       'r_jail_out', 'is_violent_recid', 'vr_charge_degree', 'vr_offense_date',
       'vr_charge_desc', 'screening_date', 'v_screening_date',
       'priors_count.1', 'end', 'event'],
      dtype='object')

In [ ]:
dataX.isnull().sum().sum() 



40268

In [ ]:
cols_null = {}
for col in dataX.columns:
  cols_null[col] = dataX[col].isna().sum()
print(cols_null)

{'compas_screening_date': 0, 'decile_score': 0, 'priors_count': 0, 'is_recid': 0, 'r_charge_degree': 3743, 'r_offense_date': 3743, 'r_charge_desc': 3801, 'r_jail_in': 4898, 'r_jail_out': 4898, 'is_violent_recid': 0, 'vr_charge_degree': 6395, 'vr_offense_date': 6395, 'vr_charge_desc': 6395, 'screening_date': 0, 'v_screening_date': 0, 'priors_count.1': 0, 'end': 0, 'event': 0}


In [ ]:
# Replace minor NaNs with 0
dataX = dataX.fillna("0")

In [ ]:
new_cols_null = {}
for col in dataX.columns:
    if dataX[col].isna().sum():
        new_cols_null[col] = dataX[col].isna().sum()
print(new_cols_null)

{}


In [ ]:
# format date strings - we will consider elapsed time from then to now / from in to out
# compas_screening_date - then to now
# screening_date: then to now
# v_screening_date: then to now
# c_jail_in/out - then (out) to now + in to out
# in_custody/out_custody - then (out) to now + in to out
then_to_now_format = ["compas_screening_date", "r_offense_date", "vr_offense_date","r_jail_in", "r_jail_out", "screening_date", "v_screening_date"]
for col in then_to_now_format:
    for i in range(len(dataX[col])):
        if not(isinstance(dataX[col][i], str)) or dataX[col][i] == "0":
            dataX[col][i] = float(dataX[col][i])
            continue
        new_date = datetime.strptime(dataX[col][i], "%Y-%m-%d").timestamp()
        dataX[col].replace({dataX[col][i]: new_date}, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
newly_processed_dataX = dataX
cols_to_encode = []
for col in dataX.columns:
    if (isinstance(dataX[col][0], str)):
        cols_to_encode.append(col)
cols_to_encode

['r_charge_degree', 'r_charge_desc', 'vr_charge_degree', 'vr_charge_desc']

In [ ]:
from sklearn.preprocessing import LabelEncoder


newly_processed_dataX[cols_to_encode] = newly_processed_dataX[cols_to_encode].apply(LabelEncoder().fit_transform)
# LabelEncoder().fit_transform(newly_processed_data["c_charge_desc"])
newly_processed_dataX



,compas_screening_date,decile_score,priors_count,is_recid,r_charge_degree,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,vr_charge_degree,vr_offense_date,vr_charge_desc,screening_date,v_screening_date,priors_count.1,end,event
0,1.357344e+09,1,0,0,10,0.0,0,0.0,0.0,0,9,0.000000e+00,0,1.357344e+09,1.357344e+09,0,444,0
1,1.364947e+09,3,1,0,10,0.0,0,0.0,0.0,0,9,0.000000e+00,0,1.364947e+09,1.364947e+09,1,1094,0
2,1.414714e+09,4,8,1,8,1437609600.0,90,0.0,0.0,0,9,0.000000e+00,0,1.414714e+09,1.414714e+09,8,265,1
3,1.390435e+09,10,1,1,7,1451865600.0,206,0.0,0.0,0,9,0.000000e+00,0,1.390435e+09,1.390435e+09,1,711,1
4,1.386547e+09,1,0,0,10,0.0,0,0.0,0.0,0,9,0.000000e+00,0,1.386547e+09,1.386547e+09,0,844,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,1.376179e+09,2,2,1,7,1397347200.0,333,1403049600.0,1403136000.0,0,9,0.000000e+00,0,1.376179e+09,1.376179e+09,2,245,1
7210,1.384906e+09,1,0,0,10,0.0,0,0.0,0.0,0,9,0.000000e+00,0,1.384906e+09,1.384906e+09,0,863,0
7211,1.390522e+09,10,13,1,7,1403827200.0,206,0.0,0.0,0,9,0.000000e+00,0,1.390522e+09,1.390522e+09,13,100,0
7212,1.377734e+09,5,5,0,10,0.0,0,0.0,0.0,0,9,0.000000e+00,0,1.377734e+09,1.377734e+09,5,323,0


In [ ]:
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

In [ ]:
from sklearn.preprocessing import StandardScaler
X = newly_processed_dataX
# y = newly_processed_dataX['label']
# X
X = np.asarray(X).astype('float32')

scaler= StandardScaler()
scaler.fit(X)
X= scaler.transform(X)
len(X[0])

18

In [ ]:
dataTrainX = X[0:5771]
dataTestX = X[5771:] # NaN
print(dataTestX)
dataTrainY = dataY[0:5771]
dataTrainY = np.asarray(dataTrainY).astype('float32')
dataTrainY.reshape(-1,1)
dataTrainY

[[ 0.9458857  -1.2287542  -0.50641406 ... -0.50641406  0.4826235
  -0.7876518 ]
 [-0.1195349   0.8719408   0.3128887  ...  0.3128887  -1.194098
  -0.7876518 ]
 [ 0.65822214 -0.87863827 -0.71123976 ... -0.71123976 -1.1289339
  -0.7876518 ]
 ...
 [ 0.66887635  1.9222882   1.9514942  ...  1.9514942  -1.1364528
  -0.7876518 ]
 [-0.1195349   0.17170915  0.3128887  ...  0.3128887  -0.5775457
  -0.7876518 ]
 [ 1.9580353  -0.17840669 -0.71123976 ... -0.71123976 -1.3294387
   1.2695966 ]]


array([0., 0., 1., ..., 1., 1., 0.], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataTrainX, dataTrainY, test_size=0.2)
x_train.shape, y_train.shape 

((4616, 18), (4616,))

In [ ]:
from sklearn import tree
t = tree.DecisionTreeClassifier()
t.fit(x_train,y_train)
y_pred = t.predict(x_test)
#Score/Accuracy
acc_decisiontree=t.score(x_test, y_test)*100
acc_decisiontree

99.39393939393939

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, t.predict(x_test))
print(f1)

0.9932497589199614


In [ ]:
preds = t.predict(dataTestX)
# preds = np.round(dataTestX)
preds = preds.astype(int)

preds.reshape((-1,1))

In [ ]:
data = pd.read_csv("participants_dataset.csv")
data = data[5771:]
data["label"] = preds
data

In [ ]:
data.to_csv("preds2_int_decisionTree2.csv")